In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s gdrive/'Team Drives'/'Data Mining Team'/ gdata

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
from tqdm import tqdm
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
import sqlite3
import pandas as pd
from textblob import TextBlob
connection = sqlite3.connect('gdata/yelpHotelData.db')
x1 = connection.execute("select * FROM review")
df = pd.read_sql("select * FROM review" ,connection)

#Select only first 10k rows.
#df = df.head(1000)

#Add sentiment analysis columns
def reviewFeatures(row):
  review = row['reviewContent']
  analysis = TextBlob(review)
  return pd.Series([len(review), analysis.sentiment.polarity, analysis.sentiment.subjectivity]
           , index = ['reviewLength', 'sentimentPolarity', 'sentimentSubjectivity'])

df[['reviewLength', 'sentimentPolarity', 'sentimentSubjectivity']]  = df.apply(reviewFeatures, axis=1)


#Convert NR->N , YR->Y
df.loc[df['flagged'] == "NR", 'flagged'] = "N"
df.loc[df['flagged'] == "YR", 'flagged'] = "Y"
df.loc[df['flagged'] == "N", 'Target'] = "0"
df.loc[df['flagged'] == "Y", 'Target'] = "1"

OperationalError: ignored

In [0]:
import re

clean_1 = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
clean_2 = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
# function to clean text data
def clean_desc(desc):
    desc = [clean_1.sub("", line.lower()) for line in desc]
    desc = [clean_2.sub(" ", line) for line in desc]
    return desc

In [0]:
df['reviewContent'] = clean_desc(df['reviewContent'])
df.head()

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,hotelID,reviewLength,sentimentPolarity,sentimentSubjectivity,Target
0,6/8/2011,MyNjnxzZVTPq,IFTr6_6NI4CgCVavIL9k5g,let me begin by saying that there are two kind...,5,18,11,28,N,tQfLGoolUMu2J0igcWcoZg,2249.0,0.025111,0.569762,0
1,8/30/2011,BdD7fsPqHQL73hwENEDT-Q,c_-hF15XgNhlyy_TqzmdaA,the only place inside the loop that you can st...,3,0,3,4,N,tQfLGoolUMu2J0igcWcoZg,261.0,-0.046875,0.625000,0
2,6/26/2009,BfhqiyfC,CiwZ6S5ZizAFL5gypf8tLA,i have walked by the tokyo hotel countless tim...,5,12,14,23,N,tQfLGoolUMu2J0igcWcoZg,1239.0,0.172028,0.615385,0
3,9/16/2010,Ol,nf3q2h-kSQoZK2jBY92FOg,if you are considering staying here watch this...,1,8,2,6,N,tQfLGoolUMu2J0igcWcoZg,80.0,0.250000,0.333333,0
4,2/5/2010,i4HIAcNTjabdpG1K4F5Q2g,Sb3DJGdZ4Rq__CqxPbae-g,this place is disgusting absolutely horrible i...,3,11,4,9,N,tQfLGoolUMu2J0igcWcoZg,251.0,-0.435000,0.505000,0


In [0]:
#Ref: https://medium.com/@makcedward/nlp-pipeline-stop-words-part-5-d6770df8a936
# Part 1 through 6
nlp = spacy.load('en_core_web_sm', disable=["tagger", "parser", "ner", "textcat"])

# tokenization using spaCy
def tokenization(x):
    desc_tokens = []
    for i in tqdm(x):
        i = nlp(i)
        temp = []
        for j in i:
            temp.append(j.text)
        desc_tokens.append(temp)
    
    return desc_tokens
  
# function to remove stopwords
def remove_stopwords(desc):
    s = []
    for r in tqdm(desc):
        s_2 = []
        for token in r:
            if nlp.vocab[token].is_stop == True:
                continue
            else:
                s_2.append(token)
        s.append(" ".join(s_2))    
        
    return s

In [0]:
#Tokenization for Train data
#Bypass for experiments

df['tokenized'] = tokenization(df['reviewContent'])
df['tokenized'] = remove_stopwords(df['tokenized'])

100%|██████████| 1000/1000 [00:00<00:00, 5319.37it/s]


In [0]:
df.head()
#S

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,hotelID,reviewLength,sentimentPolarity,sentimentSubjectivity,Target,tokenized
0,6/8/2011,MyNjnxzZVTPq,IFTr6_6NI4CgCVavIL9k5g,let me begin by saying that there are two kind...,5,18,11,28,N,tQfLGoolUMu2J0igcWcoZg,2249.0,0.025111,0.569762,0,let begin saying kinds people tokyo hotel 5 st...
1,8/30/2011,BdD7fsPqHQL73hwENEDT-Q,c_-hF15XgNhlyy_TqzmdaA,the only place inside the loop that you can st...,3,0,3,4,N,tQfLGoolUMu2J0igcWcoZg,261.0,-0.046875,0.625000,0,place inside loop stay $ 55 night place picnic...
2,6/26/2009,BfhqiyfC,CiwZ6S5ZizAFL5gypf8tLA,i have walked by the tokyo hotel countless tim...,5,12,14,23,N,tQfLGoolUMu2J0igcWcoZg,1239.0,0.172028,0.615385,0,walked tokyo hotel countless times reminds typ...
3,9/16/2010,Ol,nf3q2h-kSQoZK2jBY92FOg,if you are considering staying here watch this...,1,8,2,6,N,tQfLGoolUMu2J0igcWcoZg,80.0,0.250000,0.333333,0,considering staying watch http wwwyoutubecom...
4,2/5/2010,i4HIAcNTjabdpG1K4F5Q2g,Sb3DJGdZ4Rq__CqxPbae-g,this place is disgusting absolutely horrible i...,3,11,4,9,N,tQfLGoolUMu2J0igcWcoZg,251.0,-0.435000,0.505000,0,place disgusting absolutely horrible second st...
